In [ ]:
'''
Deep Learning Programming Assignment 1
--------------------------------------
Name: Vishnu Dutt Sharma
Roll No.: 12EC35018


======================================

Problem Statement:
Implement a simple 1 hidden layer MLP WITHOUT using any deep learning library
for predicting MNIST images. You are allowed to use linear algebra
libraries like numpy.

Resources:
1. https://ift6266h16.wordpress.com/2016/01/11/first-assignment-mlp-on-mnist/
2. https://github.com/tfjgeorge/ift6266/blob/master/notebooks/MLP.ipynb
    (In french. But the same repository has other useful ipython notebooks)

You might want to first code in an ipython notebook and later copy-paste
the code here.



======================================

Instructions:
1. Download the MNIST dataset from http://yann.lecun.com/exdb/mnist/
    (four files).
2. Extract all the files into a folder named `data' just outside
    the folder containing the main.py file. This code reads the
    data files from the folder '../data'.
3. Complete the functions in the train.py file. You might also
    create other functions for your convenience, but do not change anything
    in the main.py file or the function signatures of the train and test
    functions in the train.py file.
4. The train function must train the neural network given the training
    examples and save the in a folder named `weights' in the same
    folder as main.py
5. The test function must read the saved weights and given the test
    examples it must return the predicted labels.
6. Submit your project folder with the weights. Note: Don't include the
    data folder, which is anyway outside your project folder.

Submission Instructions:
1. Fill your name and roll no in the space provided above.
2. Name your folder in format <Roll No>_<First Name>.
    For example 12CS10001_Rohan
3. Submit a zipped format of the file (.zip only).
'''

In [1]:
import numpy as np
import os
import train

In [2]:
def load_mnist():
    data_dir = '../data'

    fd = open(os.path.join(data_dir, 'train-images.idx3-ubyte'))
    loaded = np.fromfile(file=fd, dtype=np.uint8)
    trX = loaded[16:].reshape((60000, 28, 28, 1)).astype(np.float)

    fd = open(os.path.join(data_dir, 'train-labels.idx1-ubyte'))
    loaded = np.fromfile(file=fd, dtype=np.uint8)
    trY = loaded[8:].reshape((60000)).astype(np.int)

    fd = open(os.path.join(data_dir, 't10k-images.idx3-ubyte'))
    loaded = np.fromfile(file=fd, dtype=np.uint8)
    teX = loaded[16:].reshape((10000, 28, 28, 1)).astype(np.float)

    fd = open(os.path.join(data_dir, 't10k-labels.idx1-ubyte'))
    loaded = np.fromfile(file=fd, dtype=np.uint8)
    teY = loaded[8:].reshape((10000)).astype(np.int)

    trY = np.asarray(trY)
    teY = np.asarray(teY)

    perm = np.random.permutation(trY.shape[0])
    trX = trX[perm]
    trY = trY[perm]

    perm = np.random.permutation(teY.shape[0])
    teX = teX[perm]
    teY = teY[perm]

    return trX, trY, teX, teY

In [3]:
def print_digit(digit_pixels, label='?'):
    for i in range(28):
        for j in range(28):
            if digit_pixels[i, j] > 128:
                print '#',
            else:
                print '.',
        print ''

    print 'Label: ', label


In [318]:
def softmax(vec):
    evec = np.exp(vec - np.max(vec))
    s = np.sum(evec)
    return evec/s

def sigmoid(vec):
    evec = 1 + np.exp(-vec)
    return 1/evec


def train(trainX, trainY):
    '''
    Complete this function.
    '''
    learning_rate_1 = 0.01
    learning_rate_2 = 0.01
    hidden_size = 500
    batch_size = 100
    epch = 2

    
    flat_size = trainX.shape[1]*trainX.shape[2]
    num_sample = trainX.shape[0]
    
#     hidden_layer = np.array([0]*hidden_layer_size)
    
    
    x = np.ndarray((num_sample, 1 + flat_size))

    
    for row in range(num_sample):
        x[row, 0] = 1
        x[row, 1:] = np.reshape(trainX[row], flat_size)
   
    w1 = np.load('w1_.npy')
    w2 = np.load('w2_.npy')
#     w1 = np.random.rand( hidden_size, flat_size+1)/10000
#     w2 = np.random.rand( 10, hidden_size + 1 )/10000
    

    
    for epoch in range(epch):
        
        for itr in range(trainX.shape[0]/batch_size):
#             sampleSet = range(itr*batch_size, min((itr+1)batch_size, trainX.shape[0]))
            
            batch_total = range(itr*batch_size, min((itr+1)*batch_size, trainX.shape[0]))
            
            sum_delW1 = np.zeros(w1.shape)
            sum_delW2 = np.zeros(w2.shape)
            
#             for sample in sampleSet:
            for sample in batch_total:
#                 xi = np.ndarray((1, 785))
#                 xi[0,:] = np.transpose(x[sample, :])
                batch = x[sample]
    
                sig_layer = np.apply_along_axis(sigmoid, 0, np.dot(w1, batch))
                sig_layer = np.append(1, sig_layer)
                
                out_1 = np.dot(w2, sig_layer)
                sm_layer = softmax(out_1)
                
                grad1 = sm_layer.copy()
                
                grad1[trainY[sample]] = grad1[trainY[sample]] - 1
        
                delW2 = np.dot(grad1.reshape((10,1)), sig_layer.reshape((1, hidden_size+1)))
        
#                 print grad1.shape
#                 print w2.shape
#                 print sig_layer.shape
#                 return
                temp0 = np.dot(w2.transpose(),grad1) # 801 x 1
                temp1 = sig_layer*(1-sig_layer)*temp0
        
                delW1 = np.dot(temp1.reshape((temp1.shape[0], 1)), batch.reshape((1, batch.shape[0])))
                
                sum_delW1 += delW1[1:, :]
                sum_delW2 += delW2

    #             w1 = w1 - eta_1*delW1[:, 1:]
    #             w2 = w2 - eta_2*delW2
            w1 = w1 - learning_rate_1*sum_delW1/batch_size
            w2 = w2 - learning_rate_2*sum_delW2/batch_size
            
            
        print 'Epoch: ', epoch       

        
        np.save('w1_.npy', w1)
        np.save('w2_.npy', w2)
        


In [311]:
def test(testX):
    '''
    Complete this function.
    This function must read the weight files and
    return the predicted labels.
    The returned object must be a 1-dimensional numpy array of
    length equal to the number of examples. The i-th element
    of the array should contain the label of the i-th test
    example.
    '''
    w1 = np.load('w1_.npy')
    w2 = np.load('w2_.npy')
    flat_size = testX.shape[1]*testX.shape[2]
    
    num_sample = testX.shape[0]
    
    
    x = np.ndarray((num_sample, 1 + flat_size))

    for row in range(num_sample):
        x[row, 0] = 1
        x[row, 1:] = np.reshape(testX[row], flat_size)
   
    y = np.zeros(testX.shape[0])
    
    for sample in range(num_sample):
        xi = x[sample]
        h = sigmoid(np.dot(w1, xi))
        h = np.append(1, h)
        yi = np.dot(w2, h)
        pi = softmax(yi)
        y[sample] = np.argmax(pi)
    return y

In [312]:
def main():
    trainX, trainY, testX, testY = load_mnist()
    print "Shapes: ", trainX.shape, trainY.shape, testX.shape, testY.shape

#     print "\nDigit sample"
#     print_digit(trainX[1], trainY[1])
#     print trainX[0].shape
      
#     train(trainX, trainY)
    train(trainX, trainY)
#     labels = test(testX)
#     print 'labels',labels[0]
#     print 'test',testY[0]
    labels = test(testX)
    accuracy = np.mean((labels == testY)) * 100.0
    print "\nTest accuracy: %lf%%" % accuracy

In [319]:
if __name__ == '__main__':
    main()
    

Shapes:  (60000, 28, 28, 1) (60000,) (10000, 28, 28, 1) (10000,)
Epoch:  0
Epoch:  1

Test accuracy: 96.010000%


NameError: name 'testX' is not defined